In [1]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
import subprocess

frutas_procuradas = ["LARANJA", "LIMAO", "TANGERINA"]

#Função de processamento e tratamento de exceções 
def processar_arquivo(caminho_arquivo):
    try:
        df = pd.read_excel(caminho_arquivo, skiprows=10)
        nome_tecnico = pd.read_excel(caminho_arquivo).iloc[7, 2].split(': ')[1]
        
        df = df.dropna(axis=1, how='all').drop(columns=['Unnamed: 2', 'Unnamed: 6', 'Unnamed: 7'])
        df.columns = ['Fruta', 'Atendimento Realizado', 'Atendimento Programado', 'Realizado/Programado']
        df = df.iloc[:18]
        
        df_filtrado = df[df['Fruta'].isin(frutas_procuradas)]
        df_filtrado.insert(0, 'Técnico', nome_tecnico)
        
        print(f"Processando o arquivo: {os.path.basename(caminho_arquivo)}")
        return df_filtrado
    except Exception as e:
        print(f"Erro ao processar o arquivo {os.path.basename(caminho_arquivo)}: {e}")
        return pd.DataFrame()

#Funções do Tkinter
def selecionar_diretorio_entrada():
    diretorio = filedialog.askdirectory()
    entrada_diretorio.delete(0, tk.END)
    entrada_diretorio.insert(0, diretorio)

def selecionar_diretorio_saida():
    diretorio = filedialog.askdirectory()
    entrada_diretorio_saida.delete(0, tk.END)
    entrada_diretorio_saida.insert(0, diretorio)

def processar_arquivos():
    diretorio_entrada = entrada_diretorio.get()
    diretorio_saida = entrada_diretorio_saida.get()
    nome_arquivo_saida = entrada_nome_saida.get()
    
    if not diretorio_entrada or not diretorio_saida or not nome_arquivo_saida:
        messagebox.showerror("Erro", "Por favor, selecione o diretório de entrada, o diretório de saída e o nome do arquivo de saída.")
        return
    
    # Combinar o diretório de saída e o nome do arquivo
    arquivo_saida = os.path.join(diretorio_saida, nome_arquivo_saida)
    
    if not arquivo_saida.endswith('.xlsx'):
        arquivo_saida += '.xlsx'
    
    arquivos_excel = [arq_excel for arq_excel in os.listdir(diretorio_entrada) if arq_excel.endswith(('.xlsx', '.xls'))]
    tabela_excel_final = [processar_arquivo(os.path.join(diretorio_entrada, arquivo)) for arquivo in arquivos_excel]
    
    #Saber se existe algum arquivo excel dentro da pasta
    if len(tabela_excel_final) > 0:
        todos_tecnicos_citricultura = pd.concat(tabela_excel_final, ignore_index=True)
        todos_tecnicos_citricultura.to_excel(arquivo_saida, index=False)
        
        messagebox.showinfo("Sucesso", "Processamento concluído!")
        
        # Abrir o arquivo Excel
        try:
            if os.name == 'nt':  # Windows
                os.startfile(arquivo_saida)
            elif os.name == 'posix':  # macOS e Linux
                subprocess.call(('open', arquivo_saida))
            else:
                raise OSError("Sistema operacional não suportado")
        except Exception as e:
            messagebox.showerror("Erro", f"Não foi possível abrir o arquivo: {e}")
    else:
        messagebox.showerror("Erro", "Nenhum arquivo Excel encontrado no diretório de entrada.")

# Criar a janela principal
janela = tk.Tk()
janela.title("Processador de Arquivos Excel")

# Criar e posicionar os widgets
tk.Label(janela, text="Diretório de Entrada:").grid(row=0, column=0, sticky="e")
entrada_diretorio = tk.Entry(janela, width=50)
entrada_diretorio.grid(row=0, column=1, padx=5, pady=5)
tk.Button(janela, text="Selecionar", command=selecionar_diretorio_entrada).grid(row=0, column=2)

tk.Label(janela, text="Diretório de Saída:").grid(row=1, column=0, sticky="e")
entrada_diretorio_saida = tk.Entry(janela, width=50)
entrada_diretorio_saida.grid(row=1, column=1, padx=5, pady=5)
tk.Button(janela, text="Selecionar", command=selecionar_diretorio_saida).grid(row=1, column=2)

tk.Label(janela, text="Nome do arquivo:").grid(row=2, column=0, sticky="e")
entrada_nome_saida = tk.Entry(janela, width=50)
entrada_nome_saida.grid(row=2, column=1, padx=5, pady=5)

tk.Button(janela, text="Processar Arquivos", command=processar_arquivos).grid(row=3, column=1, pady=10)

# Iniciar o loop principal da interface gráfica
janela.mainloop()
